In [1]:
import pandas as pd

import re

In [2]:
df = pd.read_csv("extracted.csv")
df

,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,Aegis Security Insurance Company,$90.00,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,$90.00,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,"Triton General Insurance Agency, LLC. - ASIC B...",NaN,No minimum premium specified in the document.,6.0,Policy period is inferred to be 6 months based...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,none
2,Affirmative,NaN,No specific minimum premium mentioned in the d...,NaN,"Policy period is not explicitly stated, but te...",Policy Fees + MVR Fee (per policy),mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none
3,Home State County Mutual Insurance Company (Ad...,NaN,No specific minimum premium mentioned in the d...,1.0,The document mentions both monthly (1-month) a...,$72,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Access General Agency Insurance Agency of Texas,NaN,No specific minimum premium mentioned in the d...,1.0,The document mentions a one month policy perio...,$10,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none
5,Clear Blue Insurance Company,NaN,No minimum premium specified in the document.,1.0,"Policies are written for One (1), Three (3) an...",$12,mandatory,mandatory,optional,optional,optional,optional,optional,none
6,"Insurance Services Office, Inc.","$2,500",The minimum limit of liability for Personal In...,6.0,Standard policy period appears to be 6 months ...,Refer to company for the appropriate fee that ...,mandatory,mandatory,optional,optional,optional,optional,optional,none
7,"Insurance Services Office, Inc.",NaN,No minimum premium information is provided in ...,NaN,No policy period information is provided in th...,NaN,none,none,none,none,none,none,none,none
8,"Insurance Services Office, Inc.",NaN,No specific minimum premium mentioned in the d...,12.0,The document mentions that the SRIP does not a...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,none
9,ISO (Insurance Services Office),NaN,No specific minimum premium mentioned in the d...,12.0,The document references a policy period in Rul...,NaN,mandatory,mandatory,mandatory,optional,optional,optional,optional,none


In [3]:
def clean_numeric(value):
    """Convert min_premium from string to float, handling cases like '$100'."""
    if isinstance(value, str):  
        value = re.sub(r"[^\d.]", "", value)  # Remove non-numeric characters except '.'
        try:
            return float(value) if value else None  # Convert to float
        except ValueError:
            return None  # Handle conversion errors
    return value  # If already a number, return as is

# Apply to the column
df["min_premium"] = df["min_premium"].apply(clean_numeric)
df["policy_period"] = df["policy_period"].apply(clean_numeric)
df["supp_fee_policy"] = df["supp_fee_policy"].apply(clean_numeric)

# Check result
df.head()


,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,Aegis Security Insurance Company,90.0,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,90.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,"Triton General Insurance Agency, LLC. - ASIC B...",NaN,No minimum premium specified in the document.,6.0,Policy period is inferred to be 6 months based...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,none
2,Affirmative,NaN,No specific minimum premium mentioned in the d...,NaN,"Policy period is not explicitly stated, but te...",NaN,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none
3,Home State County Mutual Insurance Company (Ad...,NaN,No specific minimum premium mentioned in the d...,1.0,The document mentions both monthly (1-month) a...,72.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Access General Agency Insurance Agency of Texas,NaN,No specific minimum premium mentioned in the d...,1.0,The document mentions a one month policy perio...,10.0,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none


In [4]:
# Function to get the comment corresponding to the lowest numerical value
def get_associated_comment(group, num_col, comment_col):
    """Get the comment corresponding to the lowest numerical value in the group."""
    min_idx = group[num_col].idxmin(skipna=True) 
    return group.loc[min_idx, comment_col] if pd.notna(min_idx) else None

def most_frequent_value(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# Process each group (company_name) to apply custom logic
def process_group(group):
    """Process each company group to apply merging logic."""
    result = {
        "company_name": group["company_name"].iloc[0],  # Keep company name
        "min_premium": group["min_premium"].min(skipna=True),
        "min_premium_comments": get_associated_comment(group, "min_premium", "min_premium_comments"),
        "policy_period": group["policy_period"].min(skipna=True),
        "policy_period_comments": get_associated_comment(group, "policy_period", "policy_period_comments"),
        "supp_fee_policy": group["supp_fee_policy"].min(skipna=True),
        "coverage_BI": most_frequent_value(group["coverage_BI"]),
        "coverage_PD": most_frequent_value(group["coverage_PD"]),
        "coverage_MED": most_frequent_value(group["coverage_MED"]),
        "coverage_UM_UIMBI": most_frequent_value(group["coverage_UM_UIMBI"]),
        "coverage_UMPD": most_frequent_value(group["coverage_UMPD"]),
        "coverage_COMP": most_frequent_value(group["coverage_COMP"]),
        "coverage_COLL": most_frequent_value(group["coverage_COLL"]),
        "coverage_GAP": most_frequent_value(group["coverage_GAP"]),
    }
    return pd.Series(result)

# Apply processing
df_cleaned = df.groupby("company_name").apply(process_group).reset_index(drop=True)
df_cleaned

C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_13196\2526854375.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_13196\2526854375.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_13196\2526854375.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_13196\2526854375.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In

,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,ACCC Insurance Company,NaN,None,1.0,The document mentions monthly policies as well...,2.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,Access General Agency Insurance Agency of Texas,NaN,None,1.0,The document mentions a one month policy perio...,10.0,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none
2,Accident Insurance Company,NaN,None,1.0,"Policies are written for One (1), Three (3) an...",1215.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
3,Aegis Security Insurance Company,90.0,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,90.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Affirmative,NaN,None,NaN,None,NaN,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none
5,Allied Property and Casualty Insurance Company,NaN,None,6.0,The document states that a Policy Period is si...,2.0,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,optionnal
6,American Access Casualty Company,144.0,Policy Fee on each new business policy - $144 ...,12.0,Annual policy terms are acceptable for new bus...,3.0,mandatory,mandatory,none,none,none,none,none,none
7,Auto Club County Mutual Insurance Company,NaN,None,12.0,"The document mentions ""Effective <##/##/####> ...",NaN,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none
8,Bristol West Specialty,NaN,None,6.0,The document shows policy terms of 6 and 12 mo...,NaN,mandatory,mandatory,optionnal,optionnal,optionnal,mandatory,mandatory,none
9,California Casualty Indemnity Exchange,NaN,None,6.0,Standard policy period appears to be 6 months ...,NaN,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none


In [5]:

# Save cleaned CSV file
cleaned_file_path = "./extracted_cleaned.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)

# Show result preview
df_cleaned.head()


,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,ACCC Insurance Company,NaN,None,1.0,The document mentions monthly policies as well...,2.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,Access General Agency Insurance Agency of Texas,NaN,None,1.0,The document mentions a one month policy perio...,10.0,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none
2,Accident Insurance Company,NaN,None,1.0,"Policies are written for One (1), Three (3) an...",1215.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
3,Aegis Security Insurance Company,90.0,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,90.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Affirmative,NaN,None,NaN,None,NaN,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none
